# Determine Average Ideology of Legislator by Bill Topic

This notebook combines the legislator information with the bill information to get the average ideology of legislators by bill topic. First we'll import the packages needed to perform the calculations and then we will walk through the calculation steps.

In [1]:
import pandas as pd
import requests
import numpy as np

## Prepare Data

The first step is to read in VoteView's list of bills. We restrict this to the 113th congress onwards to ensure that the average ideology calculations by bill topic reflect more current view points for each legislator. 

In [2]:

# Note we use the meta data on Bills from VoteView so that we can loop through them to get data from ProPublica
voteviewBillInfo = "https://voteview.com/static/data/out/rollcalls/HSall_rollcalls.csv"
voteviewBillDF = pd.read_csv(voteviewBillInfo, low_memory = False)
voteviewBillDF = voteviewBillDF.loc[(voteviewBillDF.congress >= 113) & (voteviewBillDF.bill_number.notna()), :]
voteviewBillDF['congress_bill_number'] = voteviewBillDF.apply(lambda x: (str(x.congress), x.bill_number), axis = 1)
voteviewBillDF.drop(['nominate_log_likelihood', 'session', 'clerk_rollnumber'], axis = 1, inplace =True)
# We get list of unique bill numbers
bills = voteviewBillDF.congress_bill_number.unique()
print('There are', len(bills), 'bills to request from the ProPublica API')


There are 2219 bills to request from the ProPublica API


Next we need to link the VoteView bill Meta data with the actual bill Votes. With a list of unique bill numbers from the 113th congress onward, it is possible to loop through the ProPublica API to request the bill information. The form of the API request is: https://api.propublica.org/congress/v1/{congress}/bills/{bill-id}.json. Note that some bill requests will fail to return information because they are for things like nominations the ProPublica tracks sepearately. These items are all outside of the realm of the topics of interest for our app, so they are excluded from our data.

In [ ]:
apiKey =  #"INSERT API KEY HERE"
baseUrl = "https://api.propublica.org/congress/v1/"

# Make request for each congress and chamber combination using list comprehension
# Remember that each bill in the list bills is a tuple where bill[0] is congressional session and bill[1] is bill number
apiRequestListBillz = [requests.get(baseUrl+bill[0]+"/bills/"+bill[1]+".json", headers = {'X-API-Key': apiKey}) 
                      for bill in bills]
        
billz = []
failed_bill_requests = []
for i,resp in enumerate(apiRequestListBillz):
    if (resp.status_code == requests.codes.ok):
        respJSON = resp.json()
        if 'results' in respJSON:
            respJSON = respJSON['results'][0]
            billz.append(respJSON)
        else: failed_bill_requests.append(i) #print('Results not returned for request', i)
    else:
        print('Request', i, 'failed')
        failed_bill_requests.append(i)
        
# We aren't currently interested in the failed requests since they are all off topic (things like nominations)      
#failed_bill_requests = bills[failed_bill_requests]
#failed_bill_df = pd.DataFrame(failed_bill_requests)
#failed_bill_df.to_csv("OUTPUT LOCATION / FILENAME.CSV HERE")

Now we just need to retain the information to recognize the bill and its primary subject. Later we'll combine this with the VoteView information on bill ideology and the individual votes.

In [ ]:
ppBillDF = pd.DataFrame(billz)
ppBillDF = ppBillDF[['congress', 'number', 'bill', 'bill_id', 'bill_slug', 'bill_type', 'primary_subject']]
ppBillDF['bill_slug'] = ppBillDF['bill_slug'].str.upper()
ppBillDF['congress'] = ppBillDF['congress'].astype('int64')

Now we can grab member votes from VoteView for ease of tabulating the results.

In [ ]:
voteviewVotes = "https://voteview.com/static/data/out/votes/HSall_votes.csv"
voteviewVoteDF = pd.read_csv(voteviewVotes)
voteviewVoteDF = voteviewVoteDF.loc[voteviewVoteDF.congress >= 113, :]
voteviewVoteDF['VotedYes'] = np.where(voteviewVoteDF.cast_code.isin([1,2,3]),  1, 0)

The final info we need before we summarize the information is to join in the bioguide_ids from VoteView so that we can join back to the ProPublica legislator data. Note that the bioguide_id corresponds to the id from ProPublica's data and represents a legislators ID for their official congresional bio.

In [ ]:
voteviewMembers = "https://voteview.com/static/data/out/members/HSall_members.csv"
voteviewMemberDF = pd.read_csv(voteviewMembers)
voteviewMemberDF = voteviewMemberDF.loc[voteviewMemberDF.congress >= 113, ['congress', 'chamber', 'icpsr', 'bioguide_id']]

Below we'll join all the bill information together.

In [ ]:
allVoteInfoDF = pd.merge(voteviewVoteDF, 
                         voteviewBillDF[['congress', 'chamber','rollnumber', 'nominate_mid_1', 'bill_number']], 
                         how = 'inner', left_on = ['congress', 'chamber','rollnumber'], 
                         right_on = ['congress', 'chamber','rollnumber'])

allVoteInfoDF = pd.merge(allVoteInfoDF, ppBillDF[['congress', 'bill_slug', 'primary_subject']],
                        how = 'left', left_on = ['congress', 'bill_number'], right_on= ['congress', 'bill_slug'])

print(allVoteInfoDF.shape)

allVoteInfoDF = pd.merge(allVoteInfoDF, voteviewMemberDF, how = 'left', 
                         left_on = ['congress', 'chamber','icpsr'], right_on = ['congress', 'chamber','icpsr'])

display(allVoteInfoDF.sample(7))

## Calculate Average Ideology by Bill Topic

This part calculates the average ideology of bills that a legislator voted "yes" on by topic.

In [ ]:
allVoteInfoDF['CondensedSubject'] = allVoteInfoDF['primary_subject']
allVoteInfoDF.loc[allVoteInfoDF.primary_subject == 'Agriculture and Food', 'CondensedSubject'] = 'AgFood'
allVoteInfoDF.loc[allVoteInfoDF.primary_subject == 'Health', 'CondensedSubject'] = 'Health'
allVoteInfoDF.loc[allVoteInfoDF.primary_subject == 'Labor and Employment', 'CondensedSubject'] = 'LaborEmployment'

allVoteInfoDF.loc[allVoteInfoDF.primary_subject.isin(['International Affairs',
                                                   'International Affairs',
                                                   'Foreign Trade and International Finance']), 
                  'CondensedSubject'] = 'DefenseGlobal'

allVoteInfoDF.loc[allVoteInfoDF.primary_subject.isin(['Energy', 'Transportation and Public Works']), 
                  'CondensedSubject'] = 'EnergyTransport'

allVoteInfoDF.loc[allVoteInfoDF.primary_subject.isin(['Finance and Financial Sector',
                                                    'Social Welfare',
                                                   'Housing and Community Development']), 
                  'CondensedSubject'] = 'Finance'


legislatorBillTopicIdeology = allVoteInfoDF.loc[allVoteInfoDF.VotedYes == 1,
                                               :].groupby(['bioguide_id', 'CondensedSubject'],
                                                    as_index=False)['nominate_mid_1'].mean()

legislatorBillTopicIdeology = legislatorBillTopicIdeology.loc[legislatorBillTopicIdeology.CondensedSubject.isin(['AgFood',
                                                                                                                'Health',
                                                                                                                'LaborEmployment',
                                                                                                                'DefenseGlobal',
                                                                                                                'EnergyTransport',
                                                                                                                'Finance']), :]
legislatorBillTopicIdeology=legislatorBillTopicIdeology.pivot(index='bioguide_id', 
                                                              columns='CondensedSubject', 
                                                              values='nominate_mid_1').reset_index()

legislatorBillTopicIdeology.fillna(0, inplace= True)

display(legislatorBillTopicIdeology.sample(7))

outputPath = "INSERT OUTPUT PATH / FILENAME.CSV HERE"
legislatorBillTopicIdeology.to_csv(outputPath)

print('Calculations complete and file is output to .CSV')